# Introduction

* MPLS VPN forwards packets based on _labels_ instead of IP addresses. It combines the best of both Overlay and Peer-to-peer model.
* Customer routes are placed in seperate VRF routing table at the PE router. 
* A VPNv4 Peering takes place between two PE router.
* PE routers also runs iBGP for building a full-mesh topology between all PE routers. 
* MPLS VPN provides a BGP free core. 

![](figs/fig_201.png)

# MPLS VPN Configuration Steps

Following steps to configure MPLS VPN 
1. Configure IGP inside the SP core.
2. Configure MPLS and LDP inside the SP core.
3. Create VRF for each customer and assign Route Distinguisher (RD)  and Route Target (RT) values.
4. Configure VPNv4 peering between both the PE routers.
5. Configure routing between PE and CE
    * Option 1: Static/Default 
    * Option 2: IGP (EIGRP, OSPF, ISIS)
    * Option 3: BGP
6. Redistribute customer routes into iBGP at the PE routers 

# Step 1 & 2: IGP and MPLS configuration at the ISP  

## Topology 
![](figs/fig_202.png)

## Service Provider configuration 
### PE-1 Router 
```
!PE1
conf t
	!! interface config 
	int f0/0
		ip add 192.168.12.1 255.255.255.0
		ip ospf network point-to-point
		no sh
	int f0/1
		ip add 192.168.14.1 255.255.255.0
		ip ospf network point-to-point
		no sh
	int l0
		ip add 1.1.1.1 255.255.255.0
		ip ospf network point-to-point
	
	!! IGP config 
	router os 1
		pass def
		router-id 1.1.1.1
		net 192.168.0.0 0.0.255.255 area 0
		net 1.1.1.1 0.0.0.0 area 0 		!for LDP
		no pass f0/0	
		no pass f0/1
	exit
	
	!! MPLS Config 
	mpls label range 100 199
	mpls ldp router-id loop0
	int range f0/0-1
		mpls ip 
	exit
end
```
### PE-2 Router 
```
PE2
conf t
	!! interface config 
	int f0/0
		ip add	192.168.34.3 255.255.255.0
		ip ospf network point-to-point
		no sh
	int f0/1
		ip add 192.168.23.3 255.255.255.0
		ip ospf network point-to-point
		no sh
	int l0
		ip add 3.3.3.3 255.255.255.0
		ip ospf network point-to-point
	
	!! IGP config 
	router os 1
		pass def
		router-id 3.3.3.3
		net 192.168.0.0 0.0.255.255 area 0
		net 3.3.3.3 0.0.0.0 area 0 		!for LDP
		no pass f0/0	
		no pass f0/1
	exit
	
	!! MPLS Config 
	mpls label range 300 399
	mpls ldp router-id loop0
	int range f0/0-1
		mpls ip 
	exit
end 
```
### P-1 Router 
```
!P1
conf t
	!! interface config 
	int f0/0
		ip add	192.168.12.2 255.255.255.0
		ip ospf network point-to-point
		no sh
	int f0/1
		ip add 192.168.23.2 255.255.255.0
		ip ospf network point-to-point
		no sh
	int l0
		ip add 2.2.2.2 255.255.255.0
		ip ospf network point-to-point
	
	!! IGP config 
	router os 1
		pass def
		router-id 2.2.2.2
		net 192.168.0.0 0.0.255.255 area 0
		net 2.2.2.2 0.0.0.0 area 0  		!for LDP
		no pass f0/0	
		no pass f0/1
	exit
	
	!! MPLS Config 
	mpls label range 200 299
	mpls ldp router-id loop0
	int range f0/0-1
		mpls ip 
	exit
end 
```
### P-2 Router
```
!P2
conf t
	!! interface config 
	int f0/0
		ip add	192.168.34.4 255.255.255.0
		ip ospf network point-to-point
		no sh
	int f0/1
		ip add 192.168.14.4 255.255.255.0
		ip ospf network point-to-point
		no sh
	int l0
		ip add 4.4.4.4 255.255.255.0
		ip ospf network point-to-point
	
	!! IGP config 
	router os 1
		pass def
		router-id 4.4.4.4
		net 192.168.0.0 0.0.255.255 area 0
		net 4.4.4.4 0.0.0.0 area 0  		!for LDP
		no pass f0/0	
		no pass f0/1
	exit
	
	!! MPLS Config 
	mpls label range 400 499
	mpls ldp router-id loop0
	int range f0/0-1
		mpls ip 
	exit
end 
```

# Step 3: VRF Configuration

## Virtual Routing and Forwarding (VRF)
Keeping all routes coming from multiple customer may cause conflict when multiuple customer advertises routes from same address range. Previously, the customer segregation was done by ACLs. However, ACL based segrigation is not scalable. VRF provides virtulizing the RIB into many logical RIBs associated with physical interfaces pointing to the respective customer. One VRF can contain multiple interfaces but one interface can't be part of multiple VRFs.
![](figs/fig_203.png)
* VRF isolates customer specific routes into a virtual routing table in a PE router alongside its global routing table.
* Logical isolation at the RIB and CEF isolates VRF FIBs at the hardware level.
* VRF solves the complexity of legacy ACL based traffic segregation. 
* Route leaking between VRFs is not allowed by default. 

## Route Distinguisher (RD) value 
![](figs/fig_204.png)

Let the PE1 receives exact same route (`10.0.0.0/24`) from three of its customer. As the customer facing interfaces are different, the router segregates the routes into their respecrive VRFs to prevent any route leaking. However, when the routes will be redistributed into the MP-BGP for VPNv4 tunnelling between PE1 and PE2. How does PE2 segrigates the routes into the corresponding VRF?

The problem is raising as when the routes are advertised into BGP the prefixes loose their uniqueness. To make the prefixes unique, additional information needs to be attached with them what helps them identifieable at the far end of the tunnel. This additional information that distingusihes customer route across the provider core is called route distingusher or RD. 

* 64 bit (8 Bytes) prepended prefix, used to convert a client's non-unique 32-bit IPv4 address into an unique $64+32=96$ bits VPNv4 address to enable transport between PE routers.
* The resultant address is called a VPNv4 address.
* RD is locally significant to a router.
* A VRF is not operational unless an RD is configured.
* RD format: `ASN:nn` or `A.B.C.D:nn`.
* Each VRF in a PE router must have an unique RD. 
 

## Route Target (RT) value

* 64-bit Extented BGP Community that is attached to a VPNv4 BGP route to indicate its VPN membership.
* Any number of RTs can be attached to a single route.
* __Export RTs__:
    * Identifies the VPN membership to which the associated VRF belongs to.
    * Are attached to a client route when converting it into a VPNv4 route.
* __Import RTs__:
    * Used to select which VPNv4 routes are to be inserted into which VRF tables.
    * On the receiving PE router, a route is imported into its VRF only if at least one RT attached to the route matches at least one import RT configured on that VRF. 